<a href="https://colab.research.google.com/github/Aggregate-Intellect/xir/blob/main/wikiQA_fineturn_converted_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
# !pip install git+https://github.com/deepset-ai/haystack.git

In [2]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, EmbeddingRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents
from haystack.pipelines import ExtractiveQAPipeline
from haystack.pipelines import DocumentSearchPipeline

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
converted_test_data = pd.read_csv("/content/original&converted_test_data.csv")

In [5]:
converted_test_data

,Unnamed: 0,question_id,question,converted_query,document_title,document_content,answer
0,0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African American immigrated to the US,African immigration to the United States,African immigration to the United States refers to immigrants to the United ...,"As such, African immigrants are to be distinguished from African American pe..."
1,1,Q4,how a water pump works,water pump works,Pump,"A small, electrically powered pump. A large, electrically driven pump (elect...","Pumps operate by some mechanism (typically reciprocating or rotary ), and co..."
2,2,Q20,how old was sue lyon when she made lolita,Sue Lyon's age when she made lolita,Lolita (1962 film),Lolita is a 1962 comedy-drama film by Stanley Kubrick based on the classic n...,"The actress who played Lolita, Sue Lyon , was fourteen at the time of filming."
3,3,Q33,how are antibodies used in,antibodies used in,antibody,Each antibody binds to a specific antigen ; an interaction similar to a lock...,"An antibody (Ab), also known as an immunoglobulin (Ig), is a large Y-shaped ..."
4,4,Q59,HOW MUCH IS CENTAVOS IN MEXICO,centavos's price in mexico,Mexican peso,The peso ( sign : $; code : MXN) is the currency of Mexico . Modern peso and...,"The peso is subdivided into 100 centavos, represented by "" ¢ ""."
...,...,...,...,...,...,...,...
238,238,Q2990,what is the main component of vaccines,NaN,vaccine,Jonas Salk in 1955 holds two bottles of a culture used to grow polio vaccine...,The agent stimulates the body's immune system to recognize the agent as fore...
239,239,Q2994,what is preciosa crystal?,NaN,Preciosa (corporation),Preciosa is the luxury brand name for the range of precision-cut lead crysta...,Preciosa is the luxury brand name for the range of precision-cut lead crysta...
240,240,Q3004,who are all of the jonas brothers,NaN,Jonas Brothers,"The Jonas Brothers are an American pop rock band. Formed in 2005, they have ...","Formed in 2005, they have gained popularity from the Disney Channel children..."
241,241,Q3008,who is mary matalin married to,NaN,Mary Matalin,"This is about the political professional. For the actress, see Marlee Matlin...",She is married to Democratic political consultant James Carville .


In [9]:
docs = []
for index, row in converted_test_data.iterrows():
    if index == 50:
        break
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

document_store.write_documents(docs)
document_store.get_document_count()

Writing Documents:   0%|          | 0/50 [00:00<?, ?it/s]

50

# Fine turning on retriver model

In [12]:
from datasets import load_dataset

wikiqa_data = load_dataset(
    'wiki_qa',
    split='test',
)

pd_data = pd.DataFrame(wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_data.shape[0]):
  last_char = pd_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_data['answer'][index] = pd_data['answer'][index] + '.'

pd_data

WARNING - datasets.builder -  Using custom data configuration default
WARNING - datasets.builder -  Reusing dataset wiki_qa (/root/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,question_id,question,document_title,answer,label
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,African immigration to the United States refers to immigrants to the United ...,0
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,The term African in the scope of this article refers to geographical or nati...,0
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,"From the Immigration and Nationality Act of 1965 to 2007, an estimated total...",0
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,African immigrants in the United States come from almost all regions in Afri...,0
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,"They include people from different national, linguistic, ethnic, racial, cul...",0
...,...,...,...,...,...
6160,Q3045,what is an open mare?,Mare,"The word can also be used for other female equine animals, particularly mule...",0
6161,Q3045,what is an open mare?,Mare,A broodmare is a mare used for breeding.,0
6162,Q3045,what is an open mare?,Mare,A horse's female parent is known as its dam.,0
6163,Q3045,what is an open mare?,Mare,"An adult male horse is called a stallion , or, if castrated , a gelding .",0


In [35]:
data = []
for question_id in pd_data['question_id'].unique().tolist():
    if len(data) >= 50:
        break
    pd_question = pd_data.loc[pd_data['question_id'] == question_id]   
    pd_neg = pd_question[pd_question['label'] == 0]
    neg_content = ' '.join(pd_neg["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1.0]
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        pd_converted_question = converted_test_data.loc[converted_test_data['question_id'] == question_id]
        question = pd_converted_question.iloc[0]["converted_query"]
        new_row = {"question": question,
                  "neg_doc": neg_content,
                  "pos_doc": answer,
                  "score": 1.0}
        data.append(new_row)
print(data)
print(len(data))

[{'question': 'African American immigrated to the US', 'neg_doc': 'African immigration to the United States refers to immigrants to the United States who are or were nationals of Africa . The term African in the scope of this article refers to geographical or national origins rather than racial affiliation. From the Immigration and Nationality Act of 1965 to 2007, an estimated total of 0.8 to 0.9 million Africans immigrated to the United States, accounting for roughly 3.3% of total immigration to the United States during this period. African immigrants in the United States come from almost all regions in Africa and do not constitute a homogeneous group. They include people from different national, linguistic, ethnic, racial, cultural and social backgrounds.', 'pos_doc': 'As such, African immigrants are to be distinguished from African American people, the latter of whom are descendants of mostly West and Central Africans who were involuntarily brought to the United States by means of t

In [36]:
fine_turned_retriever = EmbeddingRetriever(
            document_store=document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers"
)
fine_turned_retriever.train(training_data = data, learning_rate = 2e-5, n_epochs = 1, num_warmup_steps = None, batch_size = 1)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base
WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore
INFO - haystack.nodes.retriever._embedding_encoder -  GPL training/adapting SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) with 50 examples


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
from typing import Union
fine_turned_retriever.save("/content/my retriever") 

# Validation

Load test dataset

In [38]:
test_data = converted_test_data.iloc[50:100]

In [40]:
docs = []
for index, row in test_data.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

document_store.write_documents(docs)
document_store.get_document_count()

Writing Documents:   0%|          | 0/50 [00:00<?, ?it/s]

50

In [41]:
retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers",
   scale_score=False
)

document_store.update_embeddings(retriever, batch_size=128)

IR_pipeline = DocumentSearchPipeline(retriever = retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base
WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore
INFO - haystack.document_stores.faiss -  Updating embeddings for 50 docs...


Updating Embedding:   0%|          | 0/50 [00:00<?, ? docs/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# Accuracy of original model and converted query

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_correctly_retrieved_passage = 0
number_correctly_retrieved_sentence = 0


for index, row in test_data.iterrows():
    if index == 100:
        break
    query = row['converted_query']
    result = IR_pipeline.run(
             query=query, 
             params={"Retriever": {"top_k": 1}}
             )
    retrieved_document = result["documents"][0].content
    target_document = converted_test_data['document_content'][index]
    
    if retrieved_document == target_document:
        number_correctly_retrieved_passage += 1
        
        sentence_document_store = InMemoryDocumentStore()
        sentence_document_store.delete_documents()
        docs = []
        sentence_list = nltk.tokenize.sent_tokenize(retrieved_document)
        for sentence in sentence_list:
            doc = Document(content=sentence,
                           meta={"article_title": converted_test_data['document_title']})
            docs.append(doc)
            sentence_document_store.write_documents(docs)
        
        sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers",
            scale_score = False
        )

        sentence_document_store.update_embeddings(sentence_retriever)

        sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
        
        sentence_result = sentence_pipeline.run(query=query, 
                                                params={"Retriever": {"top_k": 1}})
        print("index: ", index-49)
        print(number_correctly_retrieved_passage)
        if sentence_result['documents'] != []:
            retrieved_sentence = sentence_result["documents"][0].content
        target_sentence = converted_test_data['answer'][index]
        if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
            number_correctly_retrieved_sentence += 1
            print("Found")
            print(number_correctly_retrieved_sentence)
        else:
            print("Not Found")
            print(number_correctly_retrieved_sentence)

In [43]:
print(number_correctly_retrieved_passage)
print(number_correctly_retrieved_sentence)

50
33


# Accuracy of fine-turned model and converted query

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_correctly_retrieved_passage = 0
number_correctly_retrieved_sentence = 0


for index, row in test_data.iterrows():
    if index == 100:
        break
    query = row['converted_query']
    result = IR_pipeline.run(
             query=query, 
             params={"Retriever": {"top_k": 1}}
             )
    retrieved_document = result["documents"][0].content
    target_document = converted_test_data['document_content'][index]
    
    if retrieved_document == target_document:
        number_correctly_retrieved_passage += 1
        
        sentence_document_store = InMemoryDocumentStore()
        sentence_document_store.delete_documents()
        docs = []
        sentence_list = nltk.tokenize.sent_tokenize(retrieved_document)
        for sentence in sentence_list:
            doc = Document(content=sentence,
                           meta={"article_title": converted_test_data['document_title']})
            docs.append(doc)
            sentence_document_store.write_documents(docs)
        
        sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="/content/my retriever",
            model_format="sentence_transformers",
            scale_score = False
        )

        sentence_document_store.update_embeddings(sentence_retriever)

        sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
        
        sentence_result = sentence_pipeline.run(query=query, 
                                                params={"Retriever": {"top_k": 1}})
        print("index: ", index-49)
        print(number_correctly_retrieved_passage)
        if sentence_result['documents'] != []:
            retrieved_sentence = sentence_result["documents"][0].content
        target_sentence = converted_test_data['answer'][index]
        if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
            number_correctly_retrieved_sentence += 1
            print("Found")
            print(number_correctly_retrieved_sentence)
        else:
            print("Not Found")
            print(number_correctly_retrieved_sentence)

In [32]:
print(number_correctly_retrieved_passage)
print(number_correctly_retrieved_sentence)


50
31
